In [177]:
from abc import ABC, abstractmethod
import numpy as np
import matplotlib.pyplot as plt

In [178]:
class InterfaceEnvironment(ABC):
    
    @abstractmethod
    def __init__(self):
        pass
    
    @property
    @abstractmethod
    def states(self):
        """The representation of the state space"""
        pass
    
    
    @abstractmethod
    def step(self, action):
        """Takes a step. Uses action for the step
        
        returns
        =======
        
        
        """
        pass
    
    @abstractmethod
    def reset(self):
        """resets environment. Defaults back to starting period. 
        reset method also can take a state (used for solving the model)"""
        pass
    
    

In [179]:
class EnvironmentModel1(InterfaceEnvironment):
    
    """
    Ordering of items
    states: Q, M, K, W
    shocks: epsilon, rho, psi
    """
    
    DEFAULT_AGE = 18
    DEFAULT_ASSETS = 500
    DEFAULT_KIDS = 0
    DEFAULT_WAGE = 120
    
    def __init__(self, kappa, mu_rho, sigma_rho, p_psi, sigma_epsilon, zeta, a, b, c, d):
        
        #parameters
        self.kappa = kappa
        self.mu_rho = mu_rho
        self.sigma_rho = sigma_rho
        self.p_psi = p_psi
        self.sigma_epsilon = sigma_epsilon
        self.zeta = zeta
        
        self.a = a
        self.b = b
        self.c = c
        self.d = d
        
        #states
        self.age = self.DEFAULT_AGE
        self.assets = self.DEFAULT_ASSETS
        self.kids = self.DEFAULT_KIDS
        self.wage = self.DEFAULT_WAGE
        
    def __repr__(self):
        return f"(Q: {self.age}, A: {self.assets}, K: {self.kids}, Z: {self.wage})"
    
    
    @property
    def states(self):
        return self.age, self.assets, self.kids, self.wage
    
    
    def reset(self, states=None):
        if states is not None:
            age, assets, kids, wage = states[0], states[1], states[2], states[3]
            self.age = age
            self.assets = assets
            sef.kids = kids
            self.wage = wage
        else:
            self.age = self.DEFAULT_AGE
            self.assets = self.DEFAULT_ASSETS
            self.kids = self.DEFAULT_KIDS
            self.wage = self.DEFAULT_WAGE
        
    def step(self, action, shocks=None):
        if shocks is None:
            epsilon, rho, psi = self.draw_shocks()
   
        # remember action: hours (H)
    
        ### model dynamic
        leisure = self.calc_leisure(action)
        B = self.calc_B()
        total_wage = self.calc_total_wage()
            
        salary = self.calc_salary(total_wage, action)
        
        consumption = self.calc_consumption(salary)
        
        utility = self.calc_utility(consumption, leisure, B)
        
        ### transition
        self.calc_assets(salary, consumption, rho)
        self.calc_kids(psi)
        self.calc_wage(epsilon)
        self.calc_age()
        
        # this might be changed
        dies = self.calc_dies()
        if dies is True:
            return self.states, 0, True, f'consumption: {consumption}' 
        
        return self.states, utility, False, f'consumption: {consumption}'

    
    #model dynamic
    def calc_utility(self, consumption, leisure, B):
        return consumption**(1 - B) * leisure**(B)
    
    def calc_B(self):
        return self.f_B()
    
    def calc_salary(self, total_wage, hours):
        return hours * total_wage
    
    def calc_total_wage(self):
        return self.f_w() + self.wage
    
    def calc_consumption(self, salary):
        return self.kappa * (self.assets + salary)
    
    def calc_leisure(self, hours):
        return 7 * 24 - hours
    
    def calc_dies(self):
        # stops the model (returns done flag)
        if self.age > 85:
            return True
        return False
    
    # transitions (L.O.M)
    def calc_assets(self, salary, consumption, rho):
        self.assets = rho * (self.assets + salary - consumption)
    
    def calc_kids(self, psi):
        if self.kids < 5:
            self.kids = self.kids + psi
    
    def calc_wage(self, epsilon):
        self.wage = self.zeta * self.wage * epsilon
    
    def calc_age(self):
        self.age = self.age + 1
    
    # def shocks
    def draw_shocks(self):
        return (self.draw_epsilon(), self.draw_rho(), self.draw_psi())
        
    def draw_epsilon(self):
        return np.exp(np.random.lognormal(0, self.sigma_epsilon))
    
    def draw_rho(self):
        return np.random.normal(self.mu_rho, self.sigma_rho)
    
    def draw_psi(self):
        return np.random.binomial(1,self.p_psi)
            
    
    # function
    def f_B(self):
        _exp = np.exp(self.c + self.d*self.kids)
        return self.a + self.b* ( _exp / (1 + _exp))
    
    
    def f_w(self):
        # should be changed when data is available for the permanint income structure of the income dynamic
        return 1

In [183]:
parameters = {
    'kappa' : 0.5, 
    'mu_rho' : 0.06, 
    'sigma_rho' : 0.12, 
    'p_psi' : 0.3, 
    'sigma_epsilon' : 0.1, 
    'zeta': 0.3,
    'a' : 0.3, 
    'b' : 0.3, 
    'c' : -2, 
    'd' : 1.5
}
env = EnvironmentModel1(**parameters)

In [184]:
def random_action():
    actions = [0, 15, 25, 37, 45]
    return np.random.choice(actions)

In [185]:
parameters = {
    'kappa' : 0.5, 
    'mu_rho' : 0.06, 
    'sigma_rho' : 0.12, 
    'p_psi' : 0.3, 
    'sigma_epsilon' : 0.5, 
    'zeta': 0.4,
    'a' : 0.3, 
    'b' : 0.3, 
    'c' : -2, 
    'd' : 1.5
}

env = EnvironmentModel1(**parameters)

done = False
while not done:
    action = random_action()
    _states, reward, done, _info = env.step(action)
    print('utility', reward)
    print(env, '\n')

utility 1020.2152104174124
(Q: 19, A: 301.3732884889415, K: 1, Z: 90.36818701860287) 

utility 520.4389539981368
(Q: 20, A: 194.7884894500518, K: 2, Z: 118.30558921792687) 

utility 519.842809337707
(Q: 21, A: -1.7726572590785798, K: 2, Z: 102.07284785183651) 

utility 474.49254358008545
(Q: 22, A: 213.44665837859878, K: 3, Z: 92.88619903054362) 

utility 138.6771668042678
(Q: 23, A: 0.8382926763426205, K: 3, Z: 79.80504048785222) 

utility 366.70823479554406
(Q: 24, A: 175.10008166064253, K: 3, Z: 109.1647565934851) 

utility 127.54003922288881
(Q: 25, A: -4.553845382267168, K: 3, Z: 125.58613597351993) 

utility 443.1045218196182
(Q: 26, A: 90.29799245571624, K: 3, Z: 144.2404532660379) 

utility 96.39577807268978
(Q: 27, A: -6.6496265025972034, K: 3, Z: 179.63694140808917) 

utility 539.4596054496706
(Q: 28, A: -202.59522160770604, K: 4, Z: 171.74057452774636) 

utility 421.52691609378
(Q: 29, A: 68.65804007499557, K: 5, Z: 253.44713940672028) 

utility 552.8864722123473
(Q: 30, A: 

/Users/Jeppe/.local/share/virtualenvs/speciale-P26n6twt/lib/python3.7/site-packages/ipykernel_launcher.py:90: RuntimeWarning: invalid value encountered in double_scalars


# Random Values

In [ ]:
states_history = list()
rewards_history = list()
actions_history = list()

env.reset()

for i in range(100):
    
    env.reset()
    states, actions = list(), list()
    
    done = False
    G = 0.0
    
    while not done:
        action = random_action()
        _states, reward, done, _info = env.step(action)
        print(_states, _info)
        G += reward
        states.append(_states)
        actions.append(action)
    
    states_history.append(states)
    rewards_history.append(G)
    actions_history.append(action)
    
        